In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Author: Milan Ondrasovic <milan.ondrasovic@gmail.com>

from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image, ImageStat, ImageOps

from sot.bbox import BBox
from sot.utils import Size

plt.style.use('tableau-colorblind10')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
my_model_file_path = "../model_49.pth"
my_model = torch.load(my_model_file_path, map_location=torch.device('cpu'))

new_model_file_path = "../siamfc_alexnet_e50.pth"
new_model = torch.load(new_model_file_path, map_location=torch.device('cpu'))

In [16]:
def cross_corr(
        exemplar_emb: torch.Tensor,
        instance_emb: torch.Tensor) -> torch.Tensor:
    n_exemplars = exemplar_emb.shape[0]
    n_instances, instance_c, instance_h, instance_w = instance_emb.shape

    instance_emb = instance_emb.reshape(
        -1, n_exemplars * instance_c, instance_h, instance_w)
    response_map = F.conv2d(
        input=instance_emb, weight=exemplar_emb, groups=n_exemplars)
    response_map = response_map.reshape(
        n_instances, -1, response_map.shape[-2], response_map.shape[-1])

    return response_map

def fast_xcorr(z, x):
    nz = z.size(0)
    nx, c, h, w = x.size()
    x = x.view(-1, nz * c, h, w)
    out = F.conv2d(x, z, groups=nz)
    out = out.view(nx, -1, out.size(-2), out.size(-1))
    return out

exemplar_emb = torch.ones(1, 10, 6, 6)
instance_emb = torch.ones(10, 10, 22, 22)

# my_response = cross_corr(exemplar_emb, instance_emb)
his_response = cross_corr(exemplar_emb, instance_emb)
his_response.shape

torch.Size([10, 1, 17, 17])

In [ ]:
def print_model_info(model):
    for item in model.items():
        print(f"{item[0]}: {list(item[1].shape)}")

print_model_info(my_model)